In [10]:
# Some lengthy setup.
import matplotlib.pyplot as plt
import numpy as np
import os

from exercise_code.networks.layer import (
    Sigmoid, 
    Relu, 
    LeakyRelu, 
    Tanh,
)
from exercise_code.data import (
    DataLoader,
    ImageFolderDataset,
    RescaleTransform,
    NormalizeTransform,
    FlattenTransform,
    ComposeTransform,
)
from exercise_code.data.image_folder_dataset import RandomHorizontalFlip
from exercise_code.networks import (
    ClassificationNet,
    BCE,
    CrossEntropyFromLogits
)

%load_ext autoreload
%autoreload 2
%matplotlib inline

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
download_url = "https://i2dl.vc.in.tum.de/static/data/cifar10.zip"
i2dl_exercises_path = os.path.dirname(os.path.abspath(os.getcwd()))
cifar_root = os.path.join(i2dl_exercises_path, "datasets", "cifar10")

# Use the Cifar10 mean and standard deviation computed in Exercise 3.
cifar_mean = np.array([0.49191375, 0.48235852, 0.44673872])
cifar_std  = np.array([0.24706447, 0.24346213, 0.26147554])

# Define all the transforms we will apply on the images when 
# retrieving them.
rescale_transform = RescaleTransform()
normalize_transform = NormalizeTransform(
    mean=cifar_mean,
    std=cifar_std
)
flatten_transform = FlattenTransform()
random_horizontal_flip = RandomHorizontalFlip()
compose_transform = ComposeTransform([random_horizontal_flip, rescale_transform, 
                                      normalize_transform,
                                      flatten_transform])

# Create a train, validation and test dataset.
datasets = {}
for mode in ['train', 'val', 'test']:
    crt_dataset = ImageFolderDataset(
        mode=mode,
        root=cifar_root, 
        download_url=download_url,
        transform=compose_transform,
        split={'train': 0.6, 'val': 0.2, 'test': 0.2}
    )
    datasets[mode] = crt_dataset

In [12]:
num_layer = 2
epochs = 20
reg = 0.1
batch_size = 4

# Create a dataloader for each split.
dataloaders = {}
for mode in ['train', 'val', 'test']:
    crt_dataloader = DataLoader(
        dataset=datasets[mode],
        batch_size=256,
        shuffle=True,
        drop_last=True,
    )
    dataloaders[mode] = crt_dataloader
    
num_samples = 500
overfit_dataset = ImageFolderDataset(
    mode='train',
    root=cifar_root, 
    download_url=download_url,
    transform=compose_transform,
    limit_files=num_samples
)
dataloaders['train_small'] = DataLoader(
    dataset=overfit_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
)

overfit_validset = ImageFolderDataset(
    mode='train',
    root=cifar_root, 
    download_url=download_url,
    transform=compose_transform,
    limit_files=200
)
dataloaders['val_small'] = DataLoader(
    dataset=overfit_validset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
)

In [13]:
from exercise_code.solver import Solver
from exercise_code.networks.optimizer import SGD, Adam
from exercise_code.networks import MyOwnNetwork
best_model = ClassificationNet(activation=LeakyRelu(), hidden_size=182, reg=2.0317705811992417e-06)
loss = CrossEntropyFromLogits()
solver = Solver(best_model, dataloaders['train'], dataloaders['val'], 
                learning_rate=0.00030599514043968363, loss_func=loss, activation=LeakyRelu, optimizer=Adam, patience=5)

solver.train(epochs=50)
best_model = solver.model

(Epoch 1 / 50) train loss: 2.302509; val loss: 2.302512
(Epoch 2 / 50) train loss: 1.921325; val loss: 1.747843
(Epoch 3 / 50) train loss: 1.666675; val loss: 1.641564
(Epoch 4 / 50) train loss: 1.570900; val loss: 1.578219
(Epoch 5 / 50) train loss: 1.511414; val loss: 1.545183
(Epoch 6 / 50) train loss: 1.467319; val loss: 1.509647
(Epoch 7 / 50) train loss: 1.428303; val loss: 1.493003
(Epoch 8 / 50) train loss: 1.394063; val loss: 1.475999
(Epoch 9 / 50) train loss: 1.365992; val loss: 1.447113
(Epoch 10 / 50) train loss: 1.341182; val loss: 1.453729
(Epoch 11 / 50) train loss: 1.314819; val loss: 1.434822
(Epoch 12 / 50) train loss: 1.291106; val loss: 1.420077
(Epoch 13 / 50) train loss: 1.269047; val loss: 1.413498
(Epoch 14 / 50) train loss: 1.249591; val loss: 1.407223
(Epoch 15 / 50) train loss: 1.228562; val loss: 1.405372
(Epoch 16 / 50) train loss: 1.209651; val loss: 1.392756
(Epoch 17 / 50) train loss: 1.194896; val loss: 1.390907
(Epoch 18 / 50) train loss: 1.179487; va

In [15]:
labels, pred, acc = best_model.get_dataset_prediction(dataloaders['train'])
print("Train Accuracy: {}%".format(acc*100))
labels, pred, acc = best_model.get_dataset_prediction(dataloaders['val'])
print("Validation Accuracy: {}%".format(acc*100))

Train Accuracy: 73.8147702991453%
Validation Accuracy: 52.54407051282052%


In [16]:
labels, pred, acc = best_model.get_dataset_prediction(dataloaders['test'])
print("Test Accuracy: {}%".format(acc*100))


Test Accuracy: 53.215144230769226%


In [17]:
from exercise_code.tests import save_pickle
save_pickle({"cifar_fcn": best_model}, "cifar_fcn.p")

In [20]:
from exercise_code.submit import submit_exercise

submit_exercise('exercise06')

relevant folders: ['exercise_code', 'models']
notebooks files: ['1.cifar10_classification.ipynb', 'train_my_network.ipynb']
Adding folder exercise_code
Adding folder models
Adding notebook 1.cifar10_classification.ipynb
Adding notebook train_my_network.ipynb
Zipping successful! Zip is stored under: /mnt/c/Users/Yang/Google Drive/i2dl/i2dl_exercises/exercise_06/exercise06.zip
